In [1]:
# SENSYN - Sentence Synonymity using a Neural Network

import nltk
import spacy
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
nlp = spacy.load('en_core_web_sm')

In [2]:
#Synset data structure
class SynsetStruct():
    def __init__(self, words):
        self.words = words
        self.synsets = []
        for iword in self.words:
            self.synsets.append(wn.synsets(iword))

#Feature extraction class
class FeatureExtraction:
    def __init__(self, sentence):
        self.sentence = sentence
        self.wordlist = word_tokenize(self.sentence)

    def removeStopWords(self):
        self.wordlist = [word for word in self.wordlist if word not in stopwords.words('english')]

    def getWordList(self):
        return self.wordlist

    def getPosTags(self):
        return nltk.pos_tag(self.wordlist)

    def getDependencyParseTree(self):
        return nlp(self.sentence)

    def getLemmas(self):
        doc = nlp(self.sentence)
        LemmaList = []
        for wordx in doc:
            LemmaList.append(wordx.lemma_)
        return LemmaList

    def getSynsets(self):
        syn = SynsetStruct(self.wordlist)
        return syn

    def getActiveOrPassive(self):
        beforms = ['am', 'is', 'are', 'been', 'was', 'were', 'be', 'being']
        aux = ['do', 'did', 'does', 'have', 'has', 'had']
        tokens = nltk.pos_tag(self.wordlist)
        tags = [i[1] for i in tokens]
        if tags.count('VBN') == 0:
            return "Active"
        elif tags.count('VBN') == 1 and 'been' in self.wordlist:
            return "Active"
        else:
            pos = [i for i in range(len(tags)) if tags[i] == 'VBN' and self.wordlist[i] != 'been']
            for end in pos:
                chunk = tags[:end]
                start = 0
                for i in range(len(chunk), 0, -1):
                    last = chunk.pop()
                    if last == 'NN' or last == 'PRP':
                        start = i
                        break
                sentchunk = self.wordlist[start:end]
                tagschunk = tags[start:end]
                verbspos = [i for i in range(len(tagschunk)) if tagschunk[i].startswith('V')]
                if verbspos != []:
                    for i in verbspos:
                        if sentchunk[i].lower() not in beforms and sentchunk[i].lower() not in aux:
                            break
                    else:
                        return "Passive"
        return "Active"

In [3]:
import pandas as pd
train_df = pd.read_csv("C:/Users/sidro/OneDrive/Documents/Grad/Course work/Semester 1/NLP/Project work/train-set.txt", delimiter='\t+', error_bad_lines = False)
train_df.columns = train_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
train_df.gold_tag = round(train_df.gold_tag/5.0, 2)
print(train_df)

          id                                          sentence1  \
0        s_1  But other sources close to the sale said Viven...   
1        s_2  Micron has declared its first quarterly profit...   
2        s_3  The fines are part of failed Republican effort...   
3        s_4  The American Anglican Council, which represent...   
4        s_5  The tech-loaded Nasdaq composite rose 20.96 po...   
...      ...                                                ...   
1479  s_1480  Action is needed quickly, which is why we deci...   
1480  s_1481  One could indeed wish for more and for improve...   
1481  s_1482           (Parliament accepted the oral amendment)   
1482  s_1483  - My party has serious reservations about Comm...   
1483  s_1484                                 He saw a red rose.   

                                              sentence2  gold_tag  
0     But other sources close to the sale said Viven...       0.8  
1     Micron's numbers also marked the first quarter...    

C:\Users\sidro\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [4]:
from operator import eq
pWordsSame = []
pWordsLem = []
sTagSeq = []
sActPas = []
rSame = []
rSameLem = []
rPOS = []
rHypo = []
rMero = []
rHolo = []
rHyper = []
for index,row in train_df.iterrows():
    sent1 = row['sentence1']
    sent2 = row['sentence2']
    f1 = FeatureExtraction(str(sent1))
    f2 = FeatureExtraction(str(sent2))
    pos1 = f1.getPosTags()
    pos2 = f2.getPosTags()
    if pos1 == pos2 :
        sTagSeq.append(1.0)
    else:
        sTagSeq.append(0.0)
    if f1.getActiveOrPassive() == f2.getActiveOrPassive() :
        sActPas.append(1.0)
    else:
        sActPas.append(0.0)
    dep1 = f1.getDependencyParseTree()
    dep2 = f2.getDependencyParseTree()
    root1 = [token for token in dep1 if token.head == token][0]
    root2 = [token for token in dep2 if token.head == token][0]
    if str(root1.tag_) == str(root2.tag_) :
        rPOS.append(1.0)
    else:
        rPOS.append(0.0)
    if str(root1) == str(root2) :
        rSame.append(1.0)
    else:
        rSame.append(0.0)
    temp = nlp(str(root1))
    for t in temp:
        root1lem = t.lemma_
    temp = nlp(str(root2))
    for t in temp:
        root2lem = t.lemma_
    if str(root1lem) == str(root2lem) :
        rSameLem.append(1.0)
    else:
        rSameLem.append(0.0)
    f1.removeStopWords()
    f2.removeStopWords()
    words1 = f1.getWordList()
    words2 = f2.getWordList()
    cCommonWords = len(list(set(words1).intersection(words2)))
    pWordsSame.append(round((2 * cCommonWords)/(len(words1)+len(words2)),2))
    lemms1 = f1.getLemmas()
    lemms2 = f2.getLemmas()
    cCommonLemms = len(list(set(lemms1).intersection(lemms2)))
    pWordsLem.append(round((2 * cCommonLemms)/(len(lemms1)+len(lemms2)),2))
train_df['pWordsSame'] = pWordsSame
train_df['pWordsLem'] = pWordsLem
train_df['sTagSeq'] = sTagSeq
train_df['sActPas'] = sActPas
train_df['rSame'] = rSame
train_df['rSameLem'] = rSameLem
train_df['rPOS'] = rPOS
print(train_df)

          id                                          sentence1  \
0        s_1  But other sources close to the sale said Viven...   
1        s_2  Micron has declared its first quarterly profit...   
2        s_3  The fines are part of failed Republican effort...   
3        s_4  The American Anglican Council, which represent...   
4        s_5  The tech-loaded Nasdaq composite rose 20.96 po...   
...      ...                                                ...   
1479  s_1480  Action is needed quickly, which is why we deci...   
1480  s_1481  One could indeed wish for more and for improve...   
1481  s_1482           (Parliament accepted the oral amendment)   
1482  s_1483  - My party has serious reservations about Comm...   
1483  s_1484                                 He saw a red rose.   

                                              sentence2  gold_tag  pWordsSame  \
0     But other sources close to the sale said Viven...       0.8        0.69   
1     Micron's numbers also marke

In [26]:
import tensorflow as tf
from keras import backend as K
K.tensorflow_backend._get_available_gpus()
from keras.models import Sequential
from keras.layers import Dense
X = train_df[['pWordsSame','pWordsLem','sTagSeq','rSame','rSameLem','rPOS']]
print(X)
y = train_df[['gold_tag']]
print(y)

      pWordsSame  pWordsLem  sTagSeq  rSame  rSameLem  rPOS
0           0.69       0.64      0.0    1.0       1.0   1.0
1           0.67       0.57      0.0    0.0       0.0   0.0
2           0.48       0.43      0.0    0.0       0.0   0.0
3           0.74       0.75      0.0    1.0       1.0   1.0
4           0.31       0.33      0.0    0.0       0.0   1.0
...          ...        ...      ...    ...       ...   ...
1479        0.50       0.61      0.0    0.0       0.0   0.0
1480        0.48       0.52      0.0    0.0       0.0   1.0
1481        1.00       1.00      1.0    1.0       1.0   1.0
1482        0.69       0.54      0.0    0.0       0.0   0.0
1483        0.55       0.62      0.0    0.0       0.0   1.0

[1484 rows x 6 columns]
      gold_tag
0          0.8
1          0.8
2          0.6
3          0.6
4          0.4
...        ...
1479       1.0
1480       1.0
1481       1.0
1482       1.0
1483       0.2

[1484 rows x 1 columns]


In [28]:
model = Sequential()
model.add(Dense(3, input_dim=6, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
model.fit(X, y, epochs=200, batch_size=10)

Epoch 1/200
1484/1484 [==============================] - 0s 230us/step - loss: 0.5256 - acc: 0.2817
Epoch 2/200
1484/1484 [==============================] - 0s 233us/step - loss: 0.5255 - acc: 0.2817
Epoch 3/200
1484/1484 [==============================] - 0s 231us/step - loss: 0.5254 - acc: 0.2817
Epoch 4/200
1484/1484 [==============================] - 0s 215us/step - loss: 0.5253 - acc: 0.2817
Epoch 5/200
1484/1484 [==============================] - 0s 226us/step - loss: 0.5251 - acc: 0.2817
Epoch 6/200
1484/1484 [==============================] - 0s 219us/step - loss: 0.5249 - acc: 0.2817
Epoch 7/200
1484/1484 [==============================] - 0s 219us/step - loss: 0.5249 - acc: 0.2817
Epoch 8/200
1484/1484 [==============================] - 0s 214us/step - loss: 0.5247 - acc: 0.2817
Epoch 9/200
1484/1484 [==============================] - 0s 226us/step - loss: 0.5246 - acc: 0.2817
Epoch 10/200
1484/1484 [==============================] - 0s 250us/step - loss: 0.5246 - acc: 0.2817

1484/1484 [==============================] - 0s 239us/step - loss: 0.5217 - acc: 0.2864
Epoch 82/200
1484/1484 [==============================] - 0s 234us/step - loss: 0.5216 - acc: 0.2864
Epoch 83/200
1484/1484 [==============================] - 0s 233us/step - loss: 0.5216 - acc: 0.2864
Epoch 84/200
1484/1484 [==============================] - 0s 243us/step - loss: 0.5216 - acc: 0.2864
Epoch 85/200
1484/1484 [==============================] - 0s 228us/step - loss: 0.5216 - acc: 0.2864
Epoch 86/200
1484/1484 [==============================] - 0s 236us/step - loss: 0.5217 - acc: 0.2864
Epoch 87/200
1484/1484 [==============================] - 0s 239us/step - loss: 0.5216 - acc: 0.2864
Epoch 88/200
1484/1484 [==============================] - 0s 230us/step - loss: 0.5216 - acc: 0.2864
Epoch 89/200
1484/1484 [==============================] - 0s 235us/step - loss: 0.5216 - acc: 0.2864
Epoch 90/200
1484/1484 [==============================] - 0s 244us/step - loss: 0.5216 - acc: 0.2864
Epo

Epoch 161/200
1484/1484 [==============================] - 0s 237us/step - loss: 0.5216 - acc: 0.2864
Epoch 162/200
1484/1484 [==============================] - 0s 230us/step - loss: 0.5214 - acc: 0.2864
Epoch 163/200
1484/1484 [==============================] - 0s 238us/step - loss: 0.5218 - acc: 0.2864
Epoch 164/200
1484/1484 [==============================] - 0s 238us/step - loss: 0.5217 - acc: 0.2864
Epoch 165/200
1484/1484 [==============================] - 0s 233us/step - loss: 0.5215 - acc: 0.2864
Epoch 166/200
1484/1484 [==============================] - 0s 240us/step - loss: 0.5217 - acc: 0.2864
Epoch 167/200
1484/1484 [==============================] - 0s 234us/step - loss: 0.5216 - acc: 0.2864
Epoch 168/200
1484/1484 [==============================] - 0s 240us/step - loss: 0.5217 - acc: 0.2864
Epoch 169/200
1484/1484 [==============================] - 0s 235us/step - loss: 0.5215 - acc: 0.2864
Epoch 170/200
1484/1484 [==============================] - 0s 233us/step - loss: 0